# LDA

In [79]:
import pickle

with open('./datasets/small/embeddings/e_H.pickle', 'rb') as f:
    e_H = pickle.load(f)
with open('./datasets/small/embeddings/e_A.pickle', 'rb') as f:
    e_A = pickle.load(f)
with open('./datasets/small/embeddings/e_S.pickle', 'rb') as f:
    e_S = pickle.load(f)

In [80]:
from time import time

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

def lda(embedding):
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=embedding['corpus'],
            id2word=embedding['id2word'],
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=embedding['T'], dictionary=embedding['id2word'], coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, embedding['id2word']))
        return (cs + ts) / 2

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2) # FIXME: change n to 20

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()
    return {
        'best_params': study.best_params,
        'best_trial': best_trial,
        'time': t_end - t_start,
        'model': best_lda,
    }

In [81]:
def save_models_lda(embeddings, variant):
    for name in variant:
        model = lda(embeddings[name])
        with open(f'./datasets/small/models_lda/{name}.pickle', 'wb') as f:
            pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)
        del model

In [87]:
# e_variant = ['T', 'C', 'CL', 'CLW', 'CW', 'L', 'LW', 'W']
e_variant = ['T', 'CLW']

In [88]:
# This takes 1.5 hour
save_models_lda(e_H, [f'H{t}' for t in e_variant])
# e_names = [f'A{t}' for t in e_variant]
# save_models_lda(e_A, e_names)
# e_names = [f'S{t}' for t in e_variant]
# save_models_lda(e_S, e_names)

[I 2023-08-16 09:43:38,763] A new study created in memory with name: no-name-2cef41ec-1c5f-4ff4-8c3e-d4e1607b0d53
[I 2023-08-16 09:43:45,278] Trial 0 finished with value: 0.4731796215393243 and parameters: {'num_topics': 11, 'alpha_categorical': 'symmetric', 'eta_categorical': 'scalar', 'eta': 0.619804818365397}. Best is trial 0 with value: 0.4731796215393243.
[I 2023-08-16 09:43:52,678] Trial 1 finished with value: 0.47715832043167306 and parameters: {'num_topics': 53, 'alpha_categorical': 'asymmetric', 'eta_categorical': 'scalar', 'eta': 0.24960775961266188}. Best is trial 1 with value: 0.47715832043167306.
[I 2023-08-16 09:43:56,529] A new study created in memory with name: no-name-a6a6f2ca-a3c6-49cb-98c2-3b125e303279
[I 2023-08-16 09:44:03,026] Trial 0 finished with value: 0.3330321524232395 and parameters: {'num_topics': 43, 'alpha_categorical': 'scalar', 'eta_categorical': 'scalar', 'alpha': 0.4515682238657151, 'eta': 0.2908084788933121}. Best is trial 0 with value: 0.33303215242

# BERTopic

In [84]:
import pickle

with open('./datasets/small/embeddings/e_H.pickle', 'rb') as f:
    e_H = pickle.load(f)
with open('./datasets/small/embeddings/e_A.pickle', 'rb') as f:
    e_A = pickle.load(f)
with open('./datasets/small/embeddings/e_S.pickle', 'rb') as f:
    e_S = pickle.load(f)

In [91]:
from time import time

from bertopic import BERTopic
from utils import get_topics_bertopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"
bertopic = BERTopic(embedding_model=embedding_model)

def save_models_bertopic(embeddings, variant):
    path_prefix = 'datasets/small/models_bertopic/'
    for v in variant:
        t_start = time()
        model = bertopic.fit(embeddings[v]['text'])
        t_end = time()
        model.save(
            f"{path_prefix}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        pd.DataFrame([{ 'time': t_end - t_start }]).to_csv(f'{path_prefix}{v}/time.csv', index=False)
        # print(get_topics_bertopic(model))
        del model

In [93]:
# e_variant = ['T', 'C', 'CL', 'CLW', 'CW', 'L', 'LW', 'W']
e_variant = ['T', 'CLW']
save_models_bertopic(e_H, [f'H{v}' for v in e_variant])
# save_models_bertopic(e_A, [f'A{v}' for v in e_variant])
# save_models_bertopic(e_S, [f'S{v}' for v in e_variant])